# KAGGLE MACHINE LEARNING    
## How Many Bikes?

______________________________________________________________________________________________________________________

# I/ Import data

In [1]:
import numpy as np
%pylab inline
import pandas as pd
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import GridSearchCV
from os import chdir
chdir ('/Users/flavievampouille/Documents/DS&BA/CENTRALE/Machine Learning/Kaggle/')

Populating the interactive namespace from numpy and matplotlib


### 1) Import train dataset

In [2]:
train = pd.read_csv('train.csv')

##### Data to predict

In [3]:
y = train['cnt']
del train['cnt']

##### Remove data which are not usefull

In [4]:
del train['instant']
del train['dteday']

##### Visualize data

In [6]:
train.head()

,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed
0,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0
1,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0
2,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0
3,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0
4,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0


### 2) Import test dataset

In [5]:
test = pd.read_csv('test.csv')
instantTest = test['instant']

##### Remove data which are not usefull

In [6]:
del test['instant']
del test['dteday']

##### Visualize data

In [9]:
test.head()

,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed
0,1,0,1,0,0,4,1,1,0.26,0.2273,0.56,0.3881
1,1,0,1,1,0,4,1,1,0.26,0.2727,0.56,0.0000
2,1,0,1,2,0,4,1,1,0.26,0.2727,0.56,0.0000
3,1,0,1,3,0,4,1,1,0.26,0.2576,0.56,0.1642
4,1,0,1,4,0,4,1,1,0.26,0.2576,0.56,0.1642


### 3) Cyclic data
##### Season, month, hour and weekday are obviously cyclic data

In [7]:
# season
train ['cos_season'] = cos ( 2 * pi * train['season'] / 4 )
train ['sin_season'] = sin ( 2 * pi * train['season'] / 4 )
del train ['season']

test ['cos_season'] = cos ( 2 * pi * test['season'] / 4 )
test ['sin_season'] = sin ( 2 * pi * test['season'] / 4 )
del test ['season']

# mnth
train ['cos_mnth'] = cos ( 2 * pi * train['mnth'] / 12 )
train ['sin_mnth'] = sin ( 2 * pi * train['mnth'] / 12 )
del train ['mnth']

test ['cos_mnth'] = cos ( 2 * pi * test['mnth'] / 12 )
test ['sin_mnth'] = sin ( 2 * pi * test['mnth'] / 12 )
del test ['mnth']

# hr
train ['cos_hr'] = cos ( 2 * pi * train['hr'] / 24 )
train ['sin_hr'] = sin ( 2 * pi * train['hr'] / 24 )
del train ['hr']

test ['cos_hr'] = cos ( 2 * pi * test['hr'] / 24 )
test ['sin_hr'] = sin ( 2 * pi * test['hr'] / 24 )
del test ['hr']

# weekday
train ['cos_weekday'] = cos ( 2 * pi * train['weekday'] / 7 )
train ['sin_weekday'] = sin ( 2 * pi * train['weekday'] / 7 )
del train ['weekday']

test ['cos_weekday'] = cos ( 2 * pi * test['weekday'] / 7 )
test ['sin_weekday'] = sin ( 2 * pi * test['weekday'] / 7 )
del test ['weekday']

##### Visualize new dataset

In [11]:
train.head()

,yr,holiday,workingday,weathersit,temp,atemp,hum,windspeed,cos_season,sin_season,cos_mnth,sin_mnth,cos_hr,sin_hr,cos_weekday,sin_weekday
0,0,0,0,1,0.24,0.2879,0.81,0.0,6.123234e-17,1.0,0.866025,0.5,1.000000,0.000000,0.62349,-0.781831
1,0,0,0,1,0.22,0.2727,0.80,0.0,6.123234e-17,1.0,0.866025,0.5,0.965926,0.258819,0.62349,-0.781831
2,0,0,0,1,0.22,0.2727,0.80,0.0,6.123234e-17,1.0,0.866025,0.5,0.866025,0.500000,0.62349,-0.781831
3,0,0,0,1,0.24,0.2879,0.75,0.0,6.123234e-17,1.0,0.866025,0.5,0.707107,0.707107,0.62349,-0.781831
4,0,0,0,1,0.24,0.2879,0.75,0.0,6.123234e-17,1.0,0.866025,0.5,0.500000,0.866025,0.62349,-0.781831


______________________________________________________________________________________________________________________

# II/ RandomForestRegressor

In random forests (RandomForestRegressor), each tree in the ensemble is built from a sample drawn with replacement (i.e., a bootstrap sample) from the training set. In addition, when splitting a node during the construction of the tree, the split that is chosen is no longer the best split among all features. Instead, the split that is picked is the best split among a random subset of the features. As a result of this randomness, the bias of the forest usually slightly increases (with respect to the bias of a single non-random tree) but, due to averaging, its variance also decreases, usually more than compensating for the increase in bias, hence yielding an overall better model.

##### Import method

In [12]:
from sklearn.ensemble import RandomForestRegressor

##### Choose the best parameters --> here, except the n_estimators we keep the default ones

In [13]:
RandomForestRegressor().get_params

<bound method RandomForestRegressor.get_params of RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)>

##### Estimator and score

In [14]:
estimator = RandomForestRegressor(n_estimators=10000, n_jobs=-1, verbose=True).fit(train,y)
score = cross_val_score(estimator, train, y).mean()
print("Score = %.2f" % score)

Score = 0.77


##### Calcul the predictions and export them 

In [ ]:
predict = np.round(estimator.predict(test))

final = pd.DataFrame()
final["id"] = instantTest
final["Prediction"] = predict
final.to_csv("Kaggle Cyclic RandomForestRegressor 10 000.csv",index=False)

### Results :
##### 1000 trees : Leaderboard 0.43247    
##### 10 000 trees : Leaderboard 0.43198

______________________________________________________________________________________________________________________

# III/ ExtraTreesRegressor

In extremely randomized trees (ExtraTreesRegressor), randomness goes one step further in the way splits are computed. As in random forests, a random subset of candidate features is used, but instead of looking for the most discriminative thresholds, thresholds are drawn at random for each candidate feature and the best of these randomly-generated thresholds is picked as the splitting rule. This usually allows to reduce the variance of the model a bit more, at the expense of a slightly greater increase in bias.

##### Import method

In [8]:
from sklearn.ensemble import ExtraTreesRegressor

##### Choose the best parameters --> as the previous one we only change n_estimators

In [9]:
ExtraTreesRegressor().get_params

<bound method ExtraTreesRegressor.get_params of ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
          min_samples_split=2, min_weight_fraction_leaf=0.0,
          n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
          verbose=0, warm_start=False)>

#### Estimator and score

In [10]:
estimator = ExtraTreesRegressor(n_estimators=10000, n_jobs=-1, verbose=False).fit(train,y)
score = cross_val_score(estimator, train, y).mean()
print("Score = %.2f" % score)

Score = 0.77


##### Calcul the predictions and export them 

In [ ]:
predict = np.round(estimator.predict(test))

final = pd.DataFrame()
final["id"] = instantTest
final["Prediction"] = predict
final.to_csv("Kaggle Cycle ExtraTreesRegressor 100 000.csv",index=False)

### Result :
##### 10 000 trees : Leaderboard 0.42934
### --> the best one for now

______________________________________________________________________________________________________________________

# IV/ KNeighborsRegressor

##### Import method

In [9]:
from sklearn.neighbors import KNeighborsRegressor

##### Choose the best parameters --> here we can act on parameters

In [13]:
KNeighborsRegressor().get_params

<bound method KNeighborsRegressor.get_params of KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')>

##### Estimator and score

In [10]:
estimator = KNeighborsRegressor()
param_grid = {"n_neighbors" : range(3,60,+2) , "algorithm" : ["auto", "ball_tree", "kd_tree"] }
bestEstimator = GridSearchCV(KNeighborsRegressor(), param_grid)

bestEstimator.fit(train,y)
score = cross_val_score(estimator, train, y).mean()
print("Score = %.2f" % score)

Score = 0.38


##### Best parameters

In [11]:
bestEstimator.best_params_

{'algorithm': 'auto', 'n_neighbors': 11}

##### Calcul the predictions and export them 

In [13]:
predict = np.round(bestEstimator.predict(test))

final = pd.DataFrame()
final["id"] = instantTest
final["Prediction"] = predict
final.to_csv("Kaggle Cycle KNeighborsRegressor.csv",index=False)

______________________________________________________________________________________________________________________

# V/  GradientBoostingRegressor

GradientBoosting builds an additive model in a forward stage-wise fashion; it allows for the optimization of arbitrary differentiable loss functions. In each stage a regression tree is fit on the negative gradient of the given loss function.

##### Import method